# Get historical market debt

* Query old blocks to get marketDebt for each market over time


In [1]:
import os
from copy import deepcopy
from datetime import datetime
import asyncio
import requests
import pandas as pd
import numpy as np
import json
import sqlite3
from gql import Client, gql
from gql.transport.aiohttp import AIOHTTPTransport
from decimal import Decimal
from dotenv import load_dotenv
import plotly.io as pio
import plotly.express as px
from web3 import Web3
from web3.middleware import geth_poa_middleware

load_dotenv()

pio.renderers.default = 'notebook'


In [2]:
## constants
INFURA_KEY = os.getenv('INFURA_KEY')
RPC_ENDPOINT = f'https://optimism-mainnet.infura.io/v3/{INFURA_KEY}'

# set up sqlite connection
con = sqlite3.connect("../../data/perps.db")

# get a web3 provider
w3 = Web3(Web3.HTTPProvider(RPC_ENDPOINT))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)


In [3]:
# data contract
with open('../abi/PerpsV2Data.json', 'r') as file:
    PerpsV2DataAbi = json.dumps(json.load(file))

PerpsV2DataAddress = "0xF7D3D05cCeEEcC9d77864Da3DdE67Ce9a0215A9D"


## Get the market debt

In [4]:
perpsV2Data = w3.eth.contract(address=PerpsV2DataAddress, abi=PerpsV2DataAbi)

def getMarketDebt(lookback):
    this_block = w3.eth.block_number

    if lookback == 0:
        block_lookback = this_block
    elif lookback >= 0:
        block_lookback = -1 * lookback
    else:
        raise ValueError("Lookback must be >= 0")
    
    block = w3.eth.get_block(this_block - lookback)
    
    marketSummaries = perpsV2Data.functions.allProxiedMarketSummaries().call(block_identifier=block_lookback)

    markets = [{
        'asset': market[1].decode().replace('\x00', ''),
        'marketDebt': w3.fromWei(market[7], unit='ether'),
    } for market in marketSummaries]

    df_markets = pd.DataFrame(markets)
    df_markets['marketDebt'] = df_markets['marketDebt'].astype(float)
    df_markets = df_markets.set_index('asset').transpose()
    df_markets['block'] = block['number']
    df_markets['timestamp'] = block['timestamp']

    return df_markets



In [5]:
df = pd.DataFrame()
for i in range(10):
    print(f'Running number {i}')
    df_this_block = getMarketDebt(i*100000)
    df = pd.concat([df, df_this_block], axis=0)

df

Running number 0
Running number 1
Running number 2
Running number 3
Running number 4
Running number 5
Running number 6
Running number 7
Running number 8
Running number 9


asset,sETH,sBTC,LINK,SOL,AVAX,AAVE,UNI,MATIC,APE,DYDX,...,ATOM,AXS,FLOW,FTM,NEAR,AUD,GBP,ARB,block,timestamp
marketDebt,7.326612e+06,8.338254e+06,581475.002430,210365.818190,280779.855627,3411.860543,101835.652273,459383.941671,81774.304811,212199.281399,...,20310.762373,8881.954699,5631.652249,32639.173881,3923.517734,1356.612244,2042.841854,300424.755859,84406359,1680031220
marketDebt,7.505902e+06,7.973959e+06,630954.691294,215658.919589,314260.793737,2636.294256,102058.296555,494237.160338,70588.665307,142583.758541,...,18857.813737,8907.348772,13288.306590,42970.126998,3795.676542,1343.436905,2106.064485,273701.297595,84306364,1680003775
marketDebt,7.426820e+06,8.242157e+06,549612.426504,273419.019294,329337.383724,2653.705548,102656.947397,486286.757169,72980.233729,105308.126708,...,19250.777669,3394.557403,13345.609565,44385.829889,3780.586078,1349.274314,2135.635293,74929.691476,84206373,1679978708
marketDebt,7.580817e+06,8.231462e+06,637281.603886,299455.692468,432192.179569,2512.445703,90107.993436,478491.548662,68843.803314,38336.253286,...,18907.835961,2132.040162,12311.401659,35886.403059,3226.832382,1285.289863,2254.665351,NaN,84106376,1679934642
marketDebt,7.529727e+06,8.197727e+06,646054.720992,301081.323327,436531.849172,2860.790777,91242.853023,479724.194012,84101.041477,38423.596501,...,18755.726352,1691.362129,7786.740455,24893.608669,3157.563543,1296.655494,2313.803809,NaN,84006381,1679906888
marketDebt,7.562788e+06,8.040303e+06,638977.364121,282682.181815,421643.825258,3048.875084,91243.904013,459239.514512,59105.052473,34931.157814,...,19535.141374,1379.318335,9209.395109,24622.931406,3124.969343,1296.950932,2411.670191,NaN,83906382,1679864738
marketDebt,6.737644e+06,5.934412e+06,470793.117058,469110.146314,377238.977351,3042.924591,89428.492930,457804.963890,55865.245766,24746.083579,...,25131.607540,1336.206803,8679.049313,32370.308452,3120.650321,1290.942615,2420.308326,NaN,83806383,1679830142
marketDebt,6.894270e+06,5.938789e+06,276038.955663,432550.160152,342447.646990,2984.341418,96812.894242,499350.358190,57778.053128,25547.708194,...,23668.841781,1663.537596,10520.756976,48094.052623,3185.487163,1290.295305,2424.070802,NaN,83706384,1679788280
marketDebt,7.075698e+06,6.646548e+06,229312.449630,416531.668651,361793.203445,3039.264200,80767.143770,412416.604691,62124.966107,32479.497545,...,25420.975004,741.826904,10624.410068,129493.544982,2605.074536,1290.516857,2427.785409,NaN,83606390,1679745928
marketDebt,7.626527e+06,7.027113e+06,194479.267871,439563.025665,292383.141493,2646.130751,102658.203201,363903.502628,66961.865940,37055.084090,...,22703.318386,644.848033,10730.970069,124367.563586,2559.380389,1294.176305,2431.126458,NaN,83506396,1679706902


In [6]:
assets = [c for c in df.columns if c not in ['block', 'timestamp']]
df = df[['block', 'timestamp'] + assets].sort_values('timestamp')
df

asset,block,timestamp,sETH,sBTC,LINK,SOL,AVAX,AAVE,UNI,MATIC,...,XAG,EUR,ATOM,AXS,FLOW,FTM,NEAR,AUD,GBP,ARB
marketDebt,83506396,1679706902,7.626527e+06,7.027113e+06,194479.267871,439563.025665,292383.141493,2646.130751,102658.203201,363903.502628,...,117973.436858,27275.299093,22703.318386,644.848033,10730.970069,124367.563586,2559.380389,1294.176305,2431.126458,NaN
marketDebt,83606390,1679745928,7.075698e+06,6.646548e+06,229312.449630,416531.668651,361793.203445,3039.264200,80767.143770,412416.604691,...,117999.720804,27526.103362,25420.975004,741.826904,10624.410068,129493.544982,2605.074536,1290.516857,2427.785409,NaN
marketDebt,83706384,1679788280,6.894270e+06,5.938789e+06,276038.955663,432550.160152,342447.646990,2984.341418,96812.894242,499350.358190,...,118068.679267,27719.405333,23668.841781,1663.537596,10520.756976,48094.052623,3185.487163,1290.295305,2424.070802,NaN
marketDebt,83806383,1679830142,6.737644e+06,5.934412e+06,470793.117058,469110.146314,377238.977351,3042.924591,89428.492930,457804.963890,...,118136.403195,27901.089575,25131.607540,1336.206803,8679.049313,32370.308452,3120.650321,1290.942615,2420.308326,NaN
marketDebt,83906382,1679864738,7.562788e+06,8.040303e+06,638977.364121,282682.181815,421643.825258,3048.875084,91243.904013,459239.514512,...,118192.044592,27736.418736,19535.141374,1379.318335,9209.395109,24622.931406,3124.969343,1296.950932,2411.670191,NaN
marketDebt,84006381,1679906888,7.529727e+06,8.197727e+06,646054.720992,301081.323327,436531.849172,2860.790777,91242.853023,479724.194012,...,118683.521896,29992.900356,18755.726352,1691.362129,7786.740455,24893.608669,3157.563543,1296.655494,2313.803809,NaN
marketDebt,84106376,1679934642,7.580817e+06,8.231462e+06,637281.603886,299455.692468,432192.179569,2512.445703,90107.993436,478491.548662,...,118647.062773,29561.395738,18907.835961,2132.040162,12311.401659,35886.403059,3226.832382,1285.289863,2254.665351,NaN
marketDebt,84206373,1679978708,7.426820e+06,8.242157e+06,549612.426504,273419.019294,329337.383724,2653.705548,102656.947397,486286.757169,...,117891.607214,28969.372834,19250.777669,3394.557403,13345.609565,44385.829889,3780.586078,1349.274314,2135.635293,74929.691476
marketDebt,84306364,1680003775,7.505902e+06,7.973959e+06,630954.691294,215658.919589,314260.793737,2636.294256,102058.296555,494237.160338,...,116386.794559,28410.948549,18857.813737,8907.348772,13288.306590,42970.126998,3795.676542,1343.436905,2106.064485,273701.297595
marketDebt,84406359,1680031220,7.326612e+06,8.338254e+06,581475.002430,210365.818190,280779.855627,3411.860543,101835.652273,459383.941671,...,117094.165595,27852.673689,20310.762373,8881.954699,5631.652249,32639.173881,3923.517734,1356.612244,2042.841854,300424.755859


In [7]:
df.melt(id_vars=['block', 'timestamp'], var_name='asset', value_name='market_debt')

,block,timestamp,asset,market_debt
0,83506396,1679706902,sETH,7.626527e+06
1,83606390,1679745928,sETH,7.075698e+06
2,83706384,1679788280,sETH,6.894270e+06
3,83806383,1679830142,sETH,6.737644e+06
4,83906382,1679864738,sETH,7.562788e+06
...,...,...,...,...
235,84006381,1679906888,ARB,NaN
236,84106376,1679934642,ARB,NaN
237,84206373,1679978708,ARB,7.492969e+04
238,84306364,1680003775,ARB,2.737013e+05
